In [1]:
from filterpy.common import kinematic_kf
import numpy as np
from filterpy.kalman import IMMEstimator
import math
from filterpy.kalman import KalmanFilter
import pandas as pd

In [2]:
import ipynb.fs.full.filter_base as imm_filter

In [3]:
test_df_path = "C:/Users/sakif/jupyter projects/thesis_test/modular test/Untitled Folder/STUDENTS03_data_prepared_normal3.csv"
# delta_t = 0.4

In [4]:
test_df = pd.read_csv(test_df_path, index_col=None)

In [5]:
test_df.head()

,frame_id,ped_no,pos_x,pos_y,vel_x,vel_y,rotation_angle,eta_group_x,eta_group_y,bound_x,...,n_w_y45,n_w_y46,n_w_y47,n_w_y48,n_w_y49,n_w_y50,n26_y,d_w_bound,n_w_bound_x,n_w_bound_y
0,10,0,-4.659964,-8.678696,1.470409,0.583279,0.0,0.0,0.0,1.5,...,0.0,0.0,0.0,0.0,0.0,0.0,-10.266158,11.194692,-0.416266,-0.909243
1,10,1,-1.330428,-6.608065,1.200562,0.246101,0.0,0.0,0.0,1.5,...,0.0,0.0,0.0,0.0,0.0,0.0,-10.266158,10.746638,-0.378891,-0.925441
2,10,2,-10.349845,-12.715208,1.469372,1.683016,0.0,0.0,0.0,1.5,...,0.0,0.0,0.0,0.0,0.0,0.0,-10.266158,10.317949,-0.337629,-0.941279
3,10,3,-21.047288,-12.720105,1.551914,0.815166,0.0,0.0,0.0,1.5,...,0.0,0.0,0.0,0.0,0.0,0.0,-10.266158,9.904792,-0.292733,-0.956194
4,10,4,0.241245,-6.906134,-2.211964,-0.325062,0.0,0.0,0.0,1.5,...,0.0,0.0,0.0,0.0,0.0,0.0,-10.266158,9.500195,-0.244686,-0.969602


In [6]:
def state_transition_matrix(v, yaw, T):
    term13 = (v/yaw) * (-math.cos(yaw) + math.cos(T * yaw + yaw))
    term14 = (-math.sin(yaw) + math.sin(T * yaw + yaw))/yaw
    term15 = T * v * math.cos(T * yaw + yaw)/ yaw - v * (-math.sin(yaw) + math.sin(T * yaw + yaw)) / yaw **2
    term23 = (v/yaw) * (-math.sin(yaw) + math.sin(T * yaw + yaw))
    term24 = (math.cos(yaw) - math.cos(T * yaw + yaw)) / yaw
    term25 = T * v * math.sin(T * yaw + yaw)/ yaw - v * (math.cos(yaw) - math.cos(T * yaw + yaw)) / yaw **2
    state_transition_matrix = np.array([[1, 0, term13, term14, term15],
                                        [0, 1, term23, term24, term25],
                                        [0, 0, 1, 0, T],
                                        [0, 0, 0, 1, 0],
                                        [0, 0, 0, 0, 1]])
    return state_transition_matrix

def direction(current_state, destination_state):
    # atan2(b1−a1,b2−a2)
    directions1 = np.linalg.norm((np.array(destination_state) - np.array(current_state)))
    # dest_tempp = np.array(destination_state) - np.array(current_state)
    # temp_d = math.atan2(dest_tempp[0], dest_tempp[1])
    # directions1 = math.degrees(temp_d)
    # print(directions1)
    return directions1

In [7]:
def imm_dest_filter(input_xy_temp, dest_list):
    start = input_xy_temp[0]
    # zone1, zone2, zone3, zone4 = [-2, 5.8566027], [-6.5902743, 0.065724367], [-6.5553084, 1.1867515], [15.10717, 5.5659299]
    zone1, zone2, zone3, zone4 = dest_list[0], dest_list[1], dest_list[2], dest_list[3]
    v, delta_t = 0.707, 0.4

    kf1 = KalmanFilter(dim_x=5, dim_z=2)
    kf2 = KalmanFilter(dim_x=5, dim_z=2)
    kf3 = KalmanFilter(dim_x=5, dim_z=2)
    kf4 = KalmanFilter(dim_x=5, dim_z=2)

    directions1, directions2, directions3 = direction(start, zone1), direction(start, zone2), direction(start, zone3)
    directions4 = direction(start, zone4)

    state_transition1 = state_transition_matrix(v, directions1, delta_t)
    state_transition2 = state_transition_matrix(v, directions2, delta_t)
    state_transition3 = state_transition_matrix(v, directions3, delta_t)
    state_transition4 = state_transition_matrix(v, directions4, delta_t)

    kf1.F, kf2.F, kf3.F, kf4.F = state_transition1, state_transition2, state_transition3, state_transition4

    # define initial state [x, y, yaw, velocity, yaw*dt]
    kf1.x = np.array([input_xy_temp[0, 0], input_xy_temp[0, 1], directions1, v, directions1*delta_t])
    kf2.x = np.array([input_xy_temp[0, 0], input_xy_temp[0, 1], directions2, v, directions2*delta_t])
    kf3.x = np.array([input_xy_temp[0, 0], input_xy_temp[0, 1], directions3, v, directions3*delta_t])
    kf4.x = np.array([input_xy_temp[0, 0], input_xy_temp[0, 1], directions4, v, directions4*delta_t])

    sGPS, sCourse, sVelocity, sYaw = 0.5*8.8*delta_t**2, 0.1*delta_t, 8.8*delta_t, 1.0*delta_t
    temp_q = np.diag([sGPS**2, sGPS**2, sCourse**2, sVelocity**2, sYaw**2])
    # process noise
    kf1.Q, kf2.Q, kf3.Q, kf4.Q = temp_q, temp_q, temp_q, temp_q
    # measurement noise
    kf1.R, kf2.R, kf3.R, kf4.R = np.identity(2)*5, np.identity(2)*5, np.identity(2)*5, np.identity(2)*5
    # Define the covariance matrix.
    # Here I take advantage of the fact that P already contains np.eye(dim_x), and just multiply by the uncertainty:
    kf1.P *= 10; kf2.P *= 10; kf3.P *= 10; kf4.P *= 10

    # define measurement function
    kf1.H = np.array([[1.0, 0.0, 0.0, 0.0, 0.0],
                    [0.0, 1.0, 0.0, 0.0, 0.0]])
    kf2.H = np.array([[1.0, 0.0, 0.0, 0.0, 0.0],
                    [0.0, 1.0, 0.0, 0.0, 0.0]])
    kf3.H = np.array([[1.0, 0.0, 0.0, 0.0, 0.0],
                    [0.0, 1.0, 0.0, 0.0, 0.0]])
    kf4.H = np.array([[1.0, 0.0, 0.0, 0.0, 0.0],
                    [0.0, 1.0, 0.0, 0.0, 0.0]])

    filters = [kf1, kf2, kf3, kf4]

    mu = [0.25, 0.25, 0.25, 0.25] # each filter is equally likely at the start

    trans = np.array([[0.90, 0.033, 0.033, 0.033], 
                      [0.033, 0.90, 0.033, 0.033], 
                      [0.033, 0.033, 0.90, 0.033],
                      [0.033, 0.033, 0.033, 0.90]
                     ]) 
    # M[i,j] is the probability of switching from filter j to filter i.

    imm = IMMEstimator(filters, mu, trans)
    for i in range(len(input_xy_temp)):
        z = input_xy_temp[i]
        imm.predict()
        imm.update(z)
    chosen_filter = np.argmax(imm.mu)
    estimated_destination = dest_list[chosen_filter]
    # print("Highest probability: ", np.max(imm.mu))
    return estimated_destination

In [8]:
unique_peds = np.unique(test_df['ped_no'])
dest_list = np.array(((-0.8722162, -2.3144598), (-28.848972, -14.880247), (-22.613573, -3.5443945), (1.2566633, -8.276182)))
estimated_dest_list = []
for n in range(len(unique_peds)):
    test_df_indiv3 = test_df.loc[test_df['ped_no'] == unique_peds[n]]
    input_xy_temp3 = np.array(test_df_indiv3[['pos_x', 'pos_y']])
    estimated_dest_temp = imm_dest_filter(input_xy_temp3, dest_list)
    estimated_dest_list.append(estimated_dest_temp)
    # dest_temp = input_xy_temp2[-1]
    # dest_all.append(dest_temp)
#dest_all = np.array(dest_all)
estimated_dest_list = np.array(estimated_dest_list)
len(estimated_dest_list)
# estimated_dest_list

434

In [9]:
actual_dest_list = []
for n in range(len(unique_peds)):
    test_df_indiv3 = test_df.loc[test_df['ped_no'] == unique_peds[n]]
    input_xy_temp3 = np.array(test_df_indiv3[['pos_x', 'pos_y']])
    temp_n = np.argmin(np.linalg.norm((dest_list - input_xy_temp3[-1]), axis=1))
    temp_n2 = dest_list[temp_n]
    actual_dest_list.append(temp_n2)
actual_dest_list = np.array(actual_dest_list)
len(actual_dest_list)
# actual_dest_list

434

In [10]:
np.count_nonzero(actual_dest_list[:, 1] == estimated_dest_list[:, 1])

112

In [142]:
np.count_nonzero(actual_dest_list == estimated_dest_list)

10

In [11]:
df_dest = pd.DataFrame(data=estimated_dest_list, index= unique_peds)
df_dest = df_dest.rename(columns={0: "dest_x", 1: "dest_y"})

In [12]:
df_dest

,dest_x,dest_y
0,-22.613573,-3.544395
1,-28.848972,-14.880247
2,-22.613573,-3.544395
3,1.256663,-8.276182
4,-28.848972,-14.880247
...,...,...
429,-22.613573,-3.544395
430,-22.613573,-3.544395
431,-22.613573,-3.544395
432,-28.848972,-14.880247


In [13]:
df_dest.to_csv("students03_estimated_dests1.csv", columns=["dest_x", "dest_y"])